In [4]:
from unittest.mock import patch

### Emotional journal tests

In [2]:
from web.app.assistant.emotional_journal import *
import datetime

In [3]:
# New journal update
date = date.today()
journal = '{}'
updates_count = 0
journal_date = date
gpt_model = ''

journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{"emotion": 100}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    assert result == ({"emotion": 100}, 1, date, "token usage")

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"emotion": 100}
____________________
Result:  ({'emotion': 100}, 1, datetime.date(2024, 10, 8), 'token usage')


In [4]:
#Update of exisiting journal with new and exsisting emotion
date = date.today()
journal = '{"emotion": 100}'
updates_count = 1
journal_date = date
gpt_model = ''

journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{"emotion": 50, "emotion2": 50}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    # e1 = (100 + 50) / 2 = 75
    # e2 = 50 / 2 = 25
    assert result == ({"emotion": 75, "emotion2": 25}, 2, date, 'token usage')


____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"emotion": 50, "emotion2": 50}
____________________
Result:  ({'emotion': 75.0, 'emotion2': 25.0}, 2, datetime.date(2024, 10, 8), 'token usage')


In [5]:
#Empty response fronm GPT
date = date.today()
journal = '{}'
updates_count = 0
journal_date = date
gpt_model = ''

journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("Result: ", result)
    assert result == (None, None, None, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {}
____________________
Result:  (None, None, None, 'token usage')


In [6]:
# Multiple updates of existing journal
date = date.today()
journal = '{"e": 10, "e2": 15, "e3": 62.5}'
updates_count = 2
journal_date = date
gpt_model = ''

journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{"e": 30, "e4": 5}'
chat = ['Hi']
    
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await journal.update_journal(chat)
    print("1. Result: ", result)
    journal, updates_count, journal_date, token_usege = result

mock_response = '{"e2": 20}'
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']): 
    journal = EmotionalJournal(json.dumps(journal), updates_count, journal_date, date, gpt_model)
    result = await journal.update_journal(chat)
    print("2. Result: ", result)
    journal, updates_count, journal_date, token_usege = result
    
mock_response = '{"e5": 100}'
with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']): 
    journal = EmotionalJournal(json.dumps(journal), updates_count, journal_date, date, gpt_model)
    result = await journal.update_journal(chat)
    print("3. Result: ", result)
    
# e = (10*2 + 30) / 5 = 10
# e2 (15*2 + 10) / 5 = 10
# e3 (60*2) / 5 = 25
# e4 5/5
# e5 100 /5
assert result == ({'e': 10.0, 'e2': 10.0, 'e3': 25.0, 'e4': 1.0, 'e5': 20.0}, 5, date, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e": 30, "e4": 5}
____________________
1. Result:  ({'e': 16.67, 'e2': 10.0, 'e3': 41.67, 'e4': 1.67}, 3, datetime.date(2024, 10, 8), 'token usage')
____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e2": 20}
____________________
2. Result:  ({'e': 12.5, 'e2': 12.5, 'e3': 31.25, 'e4': 1.25}, 4, datetime.date(2024, 10, 8), 'token usage')
____________________
--- Journal prompt:
 Conversation:|
Hi| 
--- Journal update response:
 {"e5": 100}
____________________
3. Result:  ({'e': 10.0, 'e2': 10.0, 'e3': 25.0, 'e4': 1.0, 'e5': 20.0}, 5, datetime.date(2024, 10, 8), 'token usage')


In [7]:
# Update journal with 1 invalid value in response
date = date.today()
journal = '{"e": 50}'
updates_count = 1
journal_date = date
gpt_model = ''

Journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{"e": -1, "e1": 10}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await Journal.update_journal(chat)
    print("Result: ", result)
    assert result == ({'e': 25.0, 'e1': 5.0}, 2, date, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"e": -1, "e1": 10}
____________________
Result:  ({'e': 25.0, 'e1': 5.0}, 2, datetime.date(2024, 10, 8), 'token usage')


In [8]:
# Update journal with all invalid values in response
date = date.today()
journal = '{"e": 10, "e2": "20"}'
updates_count = 3
journal_date = date
gpt_model = ''

Journal = EmotionalJournal(journal, updates_count, journal_date, date, gpt_model)

mock_response = '{"e": -1, "e1": 101, "e3": 100.1, "2e": -0.1}'
chat = ['Hi', 'How can I assist']

with patch('web.app.assistant.emotional_journal.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await Journal.update_journal(chat)
    print("Result: ", result)
    assert result == (None, None, None, 'token usage')

____________________
--- Journal prompt:
 Conversation:|
Hi
How can I assist| 
--- Journal update response:
 {"e": -1, "e1": 101, "e3": 100.1, "2e": -0.1}
____________________
Result:  (None, None, None, 'token usage')


### User profiler tests

In [21]:
from web.app.assistant.user_profile import *

In [22]:
# New profile change
user_profile = '[]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"0" : "Likes this", "1" : "Dislikes this"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['Likes this', 'Dislikes this'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"0" : "Likes this", "1" : "Dislikes th

In [23]:
# Update existing profile with new entires
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"2" : "loves", "3" : "hates"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['likes', 'dislikes', 'loves', 'hates'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"2" : "lo

In [24]:
# Update existing profile with new entires + changing and deleting existing entries
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"0" : "", "1" : "dislikes likes", "2": "wow new", "4": "cool"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['dislikes likes', 'wow new', 'cool'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"0" : "",

In [29]:
# Error in keys from response
user_profile = '["likes","dislikes"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"1a" : "232424", "b345" : "dislikes likes", "2": "new"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (['likes', 'dislikes', 'new'], 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "likes", "1": "dislikes"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me''
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"1a" : 

In [28]:
# Error in response format
user_profile = '["hi"]'
gpt_model = ''

profile = Profile(user_profile, gpt_model)

mock_response = '{"1": ""here is an error", "2": "correct"}'
chat = ['User: Hi', 'Assistant: How can I help']
message = "help me"

with patch('web.app.assistant.user_profile.openai_chat_request', return_value=[mock_response, 'token usage']):
    result = await profile.update_user_profile(previous_messages=chat, user_message=message)
    print("Result: ", result)
    assert result == (None, 'token usage')

- Profiler prompts:
 Your task is to update or generate a user profile from recent chat.
Provide concise entries for user preferences, likes, dislikes, and key aspects of the user's personality.
Your response should be short but descriptive, focusing on the most useful data for understanding the user.
- If no changes need to be made to the user profile, your response is empty.
- Don't duplicate the old profile; your response should only include updates.
- To change an entire entry, respond with {"number": "Changed text of this element"}
NEVER RESPOND DIRECTLY TO THE USER!
Follow this JSON format:
{
    "1": "Changed text describing the first element",
    "2": "Changed text describing the second element"
}
 PREVIOUS USER PROFILE:|
{"0": "no changes no output"}|
CHAT HISTORY:|
User: Hi
Assistant: How can I help|
Last user message: help me
Remember format: {{"number": "Changed text of this element", "number": "Changed text of this element"}}
- Generated user prifile changes: {"1": ""here